<h1 align = 'center'> Network Analysis </h1>
<h2 align = 'center'> Individual Assignment 1 </h2>

<h3 align = 'center'> LowLevel Functions </h2>

<h4 align = 'center' > @ChenYu Wang - c.wang4@students.uu.nl , 6774326 </h4>
<h4 align = 'center' > @Lena Walcher - l.t.walcher@students.uu.nl, 6774326 </h4>
<h4 align = 'center' > @Ioannis Konstantakopoulos - i.konstantakopoulos@students.uu.nl, 6774326 </h4>
<h4 align = 'center' > @Hans Franke - h.a.franke@students.uu.nl, 6774326 </h4>

Deep learning relies on a few basic operations: convolution, nonlinear activation,
pooling, and normalisation. Backpropagation of error is used to learn connection
weights, and a fully-connected layer linked to output nodes with a softmax function
are also required. So far, we have used calls to Keras’s high-level libraries to do
these operations, as these are efficiently coded and easy to use. But it is also
important that you understand the basic functions at a low level.

In [2]:
import numpy as np

## Question 20:
Write a simple function that achieves the convolution operation efficiently for twodimensional
and three-dimensional inputs. This should allow you to input a set of
convolutional filters (‘kernels’ in Keras’s terminology) and an input layer (or image)
as inputs. The input layer should have a third dimension, representing a stack of
feature maps, and each filter should have a third dimension of corresponding size.
The function should output a number of two-dimensional feature maps
corresponding to the number of input filters, though these can be stacked into a third
dimensional like the input layer. After agreeing on a common function with your
group members, show this to your teacher. (Question 20, 5 points)

In [22]:
def convolve2D(image, kernel, padding=1, strides=1):
    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print('Image Padded \n', imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return output




In [52]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt 
from functools import reduce
%matplotlib inline

class Conv2D:
    '''A Convolution layer using nxn filters.
    
    A simple function that achieves the convolution operation efficiently for two-dimensional inputs and three-dimensional inputs. 
    a set of convolutional filters (‘kernels’ in Keras’s terminology)
    an input layer (or image) as inputs. 

    The input layer should have a third dimension or two dimension, 
    representing a stack of feature maps, and each filter should have a third dimension of corresponding size. 

    The function should output a number of two-dimensional feature maps corresponding to the number of input filters, 
    though these can be stacked into a third dimensional like the input layer. 
            
    TODO: 3d
    TODO: padding
    '''

    def __init__(self, num_filters, kernal_size):
        '''
            filters is a 3 dimensions array (num_filters, 3, 3)
        '''
        self.num_filters = num_filters
        self.kernal_size = kernal_size
        self.filters = np.random.randn(num_filters, 3, 3)
        
    
    
    def iterate_regions(self, image):
        '''Generates image regions    
        ''' 
        h, w = image.shape

        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + self.kernal_size), j:(j + self.kernal_size)]
                yield im_region, i, j

    def sub_forward(self, inputs):
        '''Return a 3 dimensions array
            
        ::inputs: 28x28
        ::outputs: 26x26x8
        '''
        # (28, 28)
        h, w = inputs.shape

        # for now, padding = 0 and stride = 1 
        outputs = np.zeros((h - self.kernal_size + 1, w - self.kernal_size + 1, self.num_filters))
    
        for im_region, i, j in self.iterate_regions(inputs):
            outputs[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
        return outputs  
    
    
    def forward(self, inputs):
        if len(inputs.shape) == 2:
            return self.sub_forward(inputs)
        
        elif len(inputs.shape) == 3:
            permuted = np.transpose(inputs, (2, 0, 1))
            c, h, w = permuted.shape
            
            container = np.zeros((h - self.kernal_size + 1, w - self.kernal_size + 1, self.num_filters))

            for i in range(c):
                outputs = self.sub_forward(permuted[i])
                container += outputs
            return container     
        else:
            raise AttributeError


class Activation:
    '''Activation function Implement
    '''
    def __init__(self):
        pass

    def relu(self, in_features):
        '''A simple function that achieves rectified linear (relu) activation over a whole feature map, with a threshold at zero. 

        in_features can be numpy array, scalar, vector, or matrix
        '''
        return np.maximum(0, in_features)

    def sigmoid(self, in_features):
        '''Apply sigmoid activation function
        
        in_features can be numpy array, scalar, vector, or matrix
        '''
        return 1/(1+np.exp(-in_features))
    
    def leakyRelu(self, in_features, alpha=0.1):
        '''Apply leakyRelu activation function
        
        in_features can be numpy array, scalar, vector, or matrix
        '''
        return np.where(in_features > 0, in_features, in_features * alpha)      
    
    def softmax(self, in_features):
        '''A function that converts the activation of a 1-dimensional matrix (such as the output of a fully-connected layer) 
        into a set of probabilities that each matrix element is the most likely classification. 

        This should include the algorithmic expression of a softmax (normalised exponential) function.
        
        in_features can be numpy array, scalar, vector, or matrix
        '''
        expo = np.exp(in_features)
        expo_sum = np.sum(expo)
        return expo/expo_sum

    
class MaxPooling:
    '''Specify the spatial extent of the pooling, with the size of the output feature map changing accordingly
    '''
    def __init__(self, pool=2, stride=2):
        self.pool = pool 
        self.stride = stride 

    def iterate_regions(self, image):
        '''Generates non-overlapping kxk image regions to pool over
        '''
        h, w, c = image.shape
        
        # floor() the value
        new_h = int(np.floor(h/self.pool))
        new_w = int(np.floor(w/self.pool))
                
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * self.pool):(i * self.pool + self.stride), (j * self.pool):(j * self.pool + self.stride)]
                yield im_region, i, j

    def forward(self, inputs):
        '''Apply a forward for the maxpooling layer
        
        ::output is a 3d numpy array with dimensions (floor(h/2), floor(w/2), num_filters).
        ::input is a 3d numpy array with dimensions (h, w, num_filters)
        '''
        h, w, num_filters = inputs.shape
        
        # floor() the value
        new_h = int(np.floor(h/self.pool))
        new_w = int(np.floor(w/self.pool))
        
        output = np.zeros((new_h, new_w, num_filters))

        for im_region, i, j in self.iterate_regions(inputs):
            output[i, j] = np.amax(im_region, axis=(0, 1))

        return output
    

class Normalization():
    '''Normalization Implement
    
    TODO: for all filter or overall?
    '''
    def __init__(slef):
        self.epsilon = np.finfo(float).eps
        # self.epsilon=1e-10
        
    
    def zeromean(self,in_features):
        '''Normalisation within each feature map, modifying the feature map 
        so that its mean value is zero and its standard deviation is one.
        '''
        return (in_features - np.mean(in_features, axis=0))/ ( np.std(in_features, axis=0)+ self.epsilon )
    
    def minmax(self,in_features):
        '''min-max normalization
        '''
        return (in_features - np.amin(in_features, axis=0)) / (np.amax(in_features, axis=0)-np.amin(in_features, axis=0) + self.epsilon)
    
    def loge(self,in_features):
        '''log transform normalization
        
        note: np.log is ln, whereas np.log10 is standard base 10 log.
        '''
        return np.log(in_features)/np.log(np.amax(in_features, axis=0))
    
    def log10(self,in_features):
        '''log transform normalization
        
        note: np.log is ln, whereas np.log10 is standard base 10 log.
        '''
        return np.log10(in_features)/np.log10(np.amax(in_features, axis=0))
    
    
    
    
class FC:
    '''fully-connected layer
    specify the number of output nodes, and link each of these to every node a stack of feature maps. 
    the stack of feature maps will typically be flattened into a 1-dimensional matrix first. 
    '''
    def __init__(self, in_dim, out_dim):
        '''Divide by in_dim to reduce the variance of our initial values
        
        in_dim = Inputs Numbers of Neuron
        out_dim = Outputs Numbers of Neuron
        '''
        self.weights = np.random.randn(in_dim, out_dim) / in_dim
        self.biases = np.zeros(out_dim)

    def forward(self, inputs):
        '''Returns a 1d numpy array
        '''
        inputs = inputs.flatten()

        in_dim, out_dim = self.weights.shape

        
        outputs = np.dot(inputs, self.weights) + self.biases

        return outputs


class Softmax:
    '''A standard fully-connected layer with softmax activation.
    
    Refer https://deepai.org/machine-learning-glossary-and-terms/softmax-layer
    '''

    def __init__(self, in_dim, out_dim):
        '''Divide by in_dim to reduce the variance of our initial values
        
        in_dim = Inputs Numbers of Neuron
        out_dim = Outputs Numbers of Neuron
        '''

        self.weights = np.random.randn(in_dim, out_dim) / in_dim
        self.biases = np.zeros(out_dim)

    def forward(self, inputs):
        '''
        Performs a forward pass of the softmax layer using the given input.
        Returns a 1d numpy array containing the respective probability values.
        - input can be any array with any dimensions.
        '''
        inputs = inputs.flatten()

        in_dim, out_dim = self.weights.shape

        feature = np.dot(inputs, self.weights) + self.biases

        # softmax function
        expo = np.exp(feature)
        expo_sum = np.sum(expo, axis=0)
        out = expo / expo_sum
        
        return out

    
class clac:
    '''write some utility funtion 
    '''
    def count_dimension(inputs):
        '''count dimention
        '''
        return reduce(lambda x,y:x*y,inputs.shape)
        
    
# numpy.maximum
# Element-wise maximum of array elements.

# amax
# The maximum value of an array along a given axis, propagates NaNs.

In [32]:
kernel = np.array([[-1, 1],[-1, 1]
                 
                  ])


array_1 = np.array([[1,0,1],[1,0,1]])
array_2 =  np.array([[[ 0.,  0.,  0.,  0.],
                      [ 0.,  1.,  0.,  0.],
                      [ 0.,  0.,  0.,  0.]],

                      [[ 0.,  0.,  0.,  0.],
                      [ 0.,  1.,  0.,  0.],
                      [ 0.,  0.,  0.,  0.]]])
kernel.shape

(2, 2)

In [33]:
array_1.shape

(2, 3)

In [34]:
array_1

array([[1, 0, 1],
       [1, 0, 1]])

In [35]:
kernel

array([[-1,  1],
       [-1,  1]])

In [36]:
convolve2D(array_1, kernel, padding=0, strides=1)

array([[ 2., -2.]])

In [45]:
import keras
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255  # (50000, 32, 32, 3)
x_test = x_test.astype("float32") / 255 # (10000, 32, 32, 3)

# Need an extra dimension for colour channels
print("x train shape", x_train.shape)
print("x test shape", x_test.shape)

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes) # (50000, 10)
y_test = keras.utils.to_categorical(y_test, num_classes) # (10000, 10)

print("y train shape:", y_train.shape)
print("y test shape:", y_test.shape)

x train shape (50000, 32, 32, 3)
x test shape (10000, 32, 32, 3)
y train shape: (50000, 10)
y test shape: (10000, 10)


In [46]:
image = x_train[0]
print(image.shape)

(32, 32, 3)


In [47]:
#Check Convolution
conv = Conv2D(num_filters=8, kernal_size=3)
output = conv.forward(image)
print(output.shape)

image_reduce = image[:,:,0]
output_2d = conv.forward(image_reduce)
print(output_2d.shape)

(30, 30, 8)
(30, 30, 8)


In [48]:
#Check Max Pooling
maxpool = MaxPooling()
output = maxpool.forward(output)
print(output.shape)

(15, 15, 8)


In [53]:
#Check FC Layer
output = FC(in_dim=clac.count_dimension(output), out_dim=10).forward(output)
print(output.shape)

(10,)


In [54]:
#Check Softmax
output = Softmax(in_dim=clac.count_dimension(output), out_dim=10).forward(output)
print(output.shape)

(10,)


## Question 21:
Write a simple function that achieves rectified linear (relu) activation over a whole
feature map, with a threshold at zero. After agreeing on a common function with your
group members, show this to your teacher. (Question 21, 2 points)

## Question 22:
Write a simple function that achieves max pooling. This should allow you to specify
the spatial extent of the pooling, with the size of the output feature map changing
accordingly. After agreeing on a common function with your group members, show
this to your teacher. (Question 22, 3 points)

## Question 23:
Write a simple function that achieves normalisation within each feature map,
modifying the feature map so that its mean value is zero and its standard deviation is
one. After agreeing on a common function with your group members, show this to
your teacher. (Question 23, 2 points)

## Question 24:
Write a function that produces a fully-connected layer. This should allow you to
specify the number of output nodes, and link each of these to every node a stack of
feature maps. The stack of feature maps will typically be flattened into a 1-
dimensional matrix first. After agreeing on a common function with your group
members, show this to your teacher. (Question 24, 5 points)

## Question 25:
Write a function that converts the activation of a 1-dimensional matrix (such as the
output of a fully-connected layer) into a set of probabilities that each matrix element
is the most likely classification. This should include the algorithmic expression of a
softmax (normalised exponential) function. After agreeing on a common function with
your group members, show this to your teacher. (Question 25, 2 points)

**BONUS QUESTION:** Write a function to achieve backpropagation of error to affect
the convolutional filter (kernel) structure used in Question 20. Modify your function
from Question 20 so you can input the convolutional filters, allowing you to modify
these filters using backpropagation (outside the convolution function). Initialise the
network with random weights in the filters. After agreeing on common functions for
convolution with your group members, show your teacher how you changed this from
the answer given in Question 20. Show your teacher your code for backpropagation
(Question 26, 5 points)

**BONUS QUESTION:** Write a piece of code that uses all of these functions (from
Questions 22-28) together to make a convolutional neural network with two
convolutional layers, a fully connected layer, and an output layer (pooling is optional,
but thresholding and normalisation are required). This should give the accuracy of
the labels as an output. Show the resulting code to your teacher. (Question 27, 5
points)


**BONUS QUESTION:** Use the resulting function to learn to classify the mnist data
set, as you did in question 10. Plot the progression of the classification accuracy
over 100 cycles. Show the resulting plots to your teacher. (Question 28, 5 points).